In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv('mergedata.csv')

In [3]:
data.head()

,Usia_kandungan,Usia_ibu,Tinggi_badan,Berat_badan,Nama_Pangan,air(g),Energi(Kal),Protein(g),Lemak(g),Karbohidrat(g),Kalsium(mg),Fosfor(mg),Vit_B1(mg),Vit_B2(mg),Vit_C(mg)
0,284,27,62,100,"Bayam ,rebus",93.5,23.0,1.2,0.6,3.7,150.0,35.0,0.02,0.10,19.0
1,282,33,64,135,"Buncis, rebus",90.6,30.0,2.2,0.2,6.4,107.0,47.0,0.01,0.29,8.0
2,279,28,64,115,"Daun kacang panjang, kukus",87.2,40.0,3.7,0.3,8.0,111.0,66.0,0.15,0.20,11.0
3,282,23,67,125,"Daun singkong, rebus",90.5,31.0,3.7,0.6,4.8,160.0,50.0,0.02,0.10,34.0
4,286,25,62,93,"Daun talas, rebus",91.6,27.0,1.3,0.2,5.9,119.0,41.0,0.00,0.60,7.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Usia_kandungan  1184 non-null   int64  
 1   Usia_ibu        1184 non-null   int64  
 2   Tinggi_badan    1184 non-null   int64  
 3   Berat_badan     1184 non-null   int64  
 4   Nama_Pangan     1184 non-null   object 
 5   air(g)          1184 non-null   float64
 6   Energi(Kal)     1184 non-null   float64
 7   Protein(g)      1184 non-null   float64
 8   Lemak(g)        1184 non-null   float64
 9   Karbohidrat(g)  1184 non-null   float64
 10  Kalsium(mg)     1184 non-null   float64
 11  Fosfor(mg)      1184 non-null   float64
 12  Vit_B1(mg)      1184 non-null   float64
 13  Vit_B2(mg)      1184 non-null   float64
 14  Vit_C(mg)       1184 non-null   float64
dtypes: float64(10), int64(4), object(1)
memory usage: 138.9+ KB


In [5]:
data.isnull().sum()

Usia_kandungan    0
Usia_ibu          0
Tinggi_badan      0
Berat_badan       0
Nama_Pangan       0
air(g)            0
Energi(Kal)       0
Protein(g)        0
Lemak(g)          0
Karbohidrat(g)    0
Kalsium(mg)       0
Fosfor(mg)        0
Vit_B1(mg)        0
Vit_B2(mg)        0
Vit_C(mg)         0
dtype: int64

## **KNN**

In [6]:
from sklearn.neighbors import NearestNeighbors


In [7]:
features = ['Tinggi_badan', 'Berat_badan', 'Usia_kandungan', 'Usia_ibu', 'Nama_Pangan']
data_subset = data[features]

In [8]:
features = ['Tinggi_badan', 'Berat_badan', 'Usia_kandungan', 'Usia_ibu', 'Nama_Pangan']
data_subset = data[features].copy()
data_subset.fillna(0, inplace=True)  # Mengganti missing values dengan nilai 0 jika diperlukan


In [9]:
data_subset['Nama_Pangan'] = data_subset['Nama_Pangan'].astype('category')
data_subset['Nama_Pangan'] = data_subset['Nama_Pangan'].cat.codes


###**Inisialisasi model KNN dan melatih model**

In [10]:
k = 5  # Nilai K, yaitu jumlah tetangga terdekat yang akan digunakan
knn = NearestNeighbors(n_neighbors=k)
knn.fit(data_subset)


NearestNeighbors()

In [ ]:
user_input = [150, 60, 24, 20, 10]  # Contoh input pengguna
user_input = pd.DataFrame([user_input])  # Ubah input pengguna menjadi DataFrame
distances, indices = knn.kneighbors(user_input)


In [27]:
recommended_food = data.loc[indices[0], 'Nama_Pangan']


In [25]:
print(recommended_food)

598    Ayam goreng Kentucky,sayap
470           Beras menir, mentah
603    Ayam goreng sukabumi, dada
454              Apang kukus, kue
892                             0
Name: Nama_Pangan, dtype: object


In [18]:
import pickle
# Simpan model ke dalam file
with open('knn.h5', 'wb') as file:
    pickle.dump(knn, file)

In [20]:
num_classes = data['Nama_Pangan'].nunique()


In [21]:
print(num_classes)

1109


In [22]:
# Buat model TensorFlow
import tensorflow as tf

input_dim = 4  # Ganti dengan dimensi input yang sesuai
num_classes = 1109  # Ganti dengan jumlah kelas yang sesuai

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(input_dim, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))




In [25]:
input_shape = (None, input_dim)  # Bentuk masukan (batch_size, input_dim)
model.build(input_shape)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()



INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpvpzx9tix\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpvpzx9tix\assets


In [26]:
with open('model.tflite', 'wb') as file:
    file.write(tflite_model)
